# Exercise

In this exercise we are going to use pydantic AI with Google Gemini to analyze an image of a wire and determine if it is broken or not. Then, depending on the result, we take one of two actions: we either send an email to the QA team telling them that the wire is broken, or we don't and we ship the wire to the customer.

## Setup

First, make sure you have setup your credentials like this:

```bash
cp env.example .env
```

Then open .env and insert your API key for Gemini, like this:

```yaml
GEMINI_API_KEY=[your key]
...
```
while leaving the rest untouched.

Then we can load the credentials like this:

In [ ]:
from dotenv import load_dotenv
load_dotenv()

Since Pydantic AI uses asyncio (asyncronous code), we need to do use nest_asyncio to be able to work in Jupyter:

**NOTE**: this is not necessary if you are writing scripts. Also, if you know how to use asyncio in python, you can use the async version of pydantic AI (i.e., change all the calls to `run_sync` to `await run(...)` in the following code and you can then insert it in your async codebase)

In [ ]:
import nest_asyncio

# TODO: call nest_asyncio.apply to make sure you can use Pydantic AI
# in jupyter
... # complete

Let's first create some utilities that we will use later:

In [ ]:

from PIL import Image
import matplotlib.pyplot as plt
    

def display_image_with_caption(image: Image.Image, caption: str):
    plt.figure(figsize=(10, 6))
    plt.imshow(image)
    plt.axis('off')
    plt.title(caption, wrap=True, fontsize=8, pad=20)
    plt.tight_layout()
    plt.show()

## Image analysis with Pydantic AI and Gemini

Let's use Pydantic AI and Gemini to do some image analysis.

In [ ]:

from pydantic import BaseModel
from typing import Literal
from pydantic_ai import Agent
from pydantic_ai.messages import BinaryContent
from io import BytesIO


# TODO: write a pydantic model describing the schema of the output.
# Include a docstring with our prompt:
# "Detect whether the write is intact or not, and describe the picture."
# The output schema should have the following elements:
# * `status`, which can only take the values "intact", "not_intact"
# * `description` with a type of str
class ImageAnalysis(BaseModel):
    """
    Detect whether the write is intact or not, and describe the picture.
    """
    status: Literal[...]  # complete
    description: ... # complete


# TODO: Create agent with structured output
# Use the google-gla:gemini-2.5-flash model and a relevant system prompt.
agent = Agent(
    ...,  # complete
    output_type=...,  # complete
    system_prompt=... # complete
)


def analyze_image_with_structured_output(img: Image.Image) -> ImageAnalysis:
    
    # Limit size to reduce token usage
    img.thumbnail((600, 600))
    
    # Encode image in a BinaryContent instance
    image_bytes = BytesIO()
    img.save(image_bytes, format='JPEG')
    binary_content = BinaryContent(data=image_bytes.getvalue(), media_type='image/jpeg')
    
    # TODO: run analysis by calling agent.run_sync and providing the binary_content
    # HINT: rememebr to pass in a list (of one element in this case)
    result = ... # complete
    
    return result.output

# Analyze image with structured output

# TODO: load the "wire.jpg" image using pillow
image = Image.open(...)  # complete
# TODO: run the analysis using the function we just wrote
# (analyze_image_with_structured_output)
analysis = ... # complete

# The output is an instance of our output model. We can pretty-print it
# using json
print("=== Image Analysis Results ===")
print(analysis.model_dump_json(indent=4))

display_image_with_caption(image, analysis.model_dump_json(indent=4))


In [ ]:
def send_alert_to_qa_team(description: str):
    email = f"""
    From: automatic quality assurance

    The wire looks broken. Report:

    {description}
    """
    # We don't really send an email
    print(email)


def mark_for_shipping():
    # ...
    pass

# TODO: write some if/else python code to implement the following
# logic: if the wire is not_intact, call the send_alert_to_qa_team function
# passing analysis.description to it,
# otherwise call mark_for_shipping
... # complete